In [3]:
%pylab
import numpy as np
import pandas as pd

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [4]:
test_data = pd.read_csv("data/test_data.csv")
train_data = pd.read_csv("data/train_data.csv")

In [5]:
train_matrix = pd.pivot_table(train_data, values='quantity', 
                              index=['customer_id'], 
                              columns=['item_id'],
                             aggfunc=np.sum,
                             fill_value=0)   #change train from dataframe to matrix form
# row is customer, column is item and then the value is aggregated sum of quantity

In [6]:
train_matrix.shape

(4889, 3365)

In [234]:
test_data.shape

(4970, 6)

In [218]:
test_data.iloc[0]["customer_id"]

'000005_APM'

In [235]:
test_user_not_in_train = []
for index in test_data.index:
    if test_data.iloc[index]["customer_id"] not in train_matrix.index:
        test_user_not_in_train.append(index)

In [236]:
test_data = test_data.drop(test_data.index[test_user_not_in_train])

In [237]:
test_data.shape

(2024, 6)

In [238]:
test_data = test_data.reset_index(drop=True)

In [62]:
len(test_user_not_in_train)

0

In [7]:
test_matrix = pd.pivot_table(test_data, values='quantity', index=['customer_id'], columns=['item_id'], aggfunc=np.sum, fill_value=0)

In [246]:
item_id_array = test_data.as_matrix(columns=["item_id"])

In [251]:
item_id_array = np.unique(item_id_array)

In [8]:
test_matrix.shape

(437, 578)

In [254]:
test_item_not_in_train = []
for column in item_id_array.tolist():
    if column not in train_matrix.columns:
        test_item_not_in_train.append(column)

In [279]:
test_data.to_csv("data/test_data.csv", index=False)

(437, 578)

In [284]:
test_matrix.shape

(437, 578)

In [11]:
R_t = test_matrix.values

In [12]:
test_nonzero_index = np.nonzero(R_t)
len(test_nonzero_index[0])

1025

In [13]:
test_nonzero_index

(array([  0,   0,   1, ..., 435, 436, 436]),
 array([390, 417, 407, ..., 556, 414, 551]))

In [178]:
np.unique(R_t[test_nonzero_index], return_counts=True)

(array([1, 2, 3, 4]), array([988,  35,   1,   1]))

In [85]:
user_id = []
item_id = []
for i in xrange(1025):
    user_id.append(test_matrix.index[test_nonzero_index[0][i]])
    item_id.append(test_matrix.columns[test_nonzero_index[1][i]])

In [14]:
user_id_index_in_train = []
item_id_index_in_train = []
for i in xrange(1025):
    user_id_index_in_train.append(np.where(train_matrix.index == test_matrix.index[test_nonzero_index[0][i]])[0][0])
    item_id_index_in_train.append(np.where(train_matrix.columns == test_matrix.columns[test_nonzero_index[1][i]])[0][0])

In [15]:
len(user_id_index_in_train)

1025

In [103]:
len(item_id_index_in_train)

1025

In [228]:
test_matrix = test_matrix[(test_matrix.T != 0).any()]   # drop the row with all zeros

In [115]:
len(np.unique(user_id_index_in_train)) == test_matrix.shape[0]

True

In [119]:
test_matrix.to_csv("data/test_data.csv")

In [122]:
test = pd.read_csv("data/test_data.csv", index_col="customer_id")

In [73]:
## now construct the predict matrix
X = np.loadtxt("data/X.csv", delimiter=",")
Y = np.loadtxt("data/Y.csv", delimiter=",")
P_hat = np.dot(X, Y.T)
P_hat.shape

(4889, 3365)

In [74]:
P_hat_sort = np.argsort(P_hat)

In [75]:
rank_ui = []
for i in xrange(len(item_id_index_in_train)):
    rank_ui.append(np.where(P_hat_sort[user_id_index_in_train[i]][::-1] == item_id_index_in_train[i])[0][0])

In [17]:
r_ui = []
for i in xrange(len(item_id_index_in_train)):
    r_ui.append(R_t[test_nonzero_index[0][i]][test_nonzero_index[1][i]])

In [77]:
n = P_hat.shape[1]

In [78]:
rank_ui = np.true_divide(rank_ui, n)

In [204]:
np.sum(np.multiply(rank_ui, r_ui)) / np.sum(r_ui)

0.31285687378356319

In [205]:
np.average(rank_ui, weights=r_ui)

0.31285687378356319

In [208]:
test = pd.read_csv("data/test_data.csv", index_col="customer_id")

In [28]:
train_matrix.shape

(4889, 3365)

In [18]:
## popularity based
popularity = np.sum(train_matrix.values, axis=0)
popularity.shape

(3365,)

In [19]:
popularity_rank = np.argsort(popularity)[::-1]

In [21]:
rank_ui_popularity = []
for i in xrange(len(item_id_index_in_train)):
    rank_ui_popularity.append(np.where(popularity_rank == item_id_index_in_train[i])[0][0])

In [25]:
rank_ui_popularity = np.true_divide(rank_ui_popularity, train_matrix.shape[1])

In [27]:
np.average(rank_ui_popularity, weights=r_ui)

0.36065099861177963

In [29]:
## neighbor based
R = train_matrix.values

In [38]:
from sklearn.metrics.pairwise import pairwise_distances

In [42]:
P_neighbor= pairwise_distances(R.T, metric="cosine")

In [58]:
S_n = np.subtract(1, P_neighbor)

In [59]:
S_n.shape

(3365, 3365)

In [62]:
P_n = np.dot(R, S_n)

In [66]:
P_n_sort = np.argsort(P_n)

In [67]:
rank_neighbor_ui = []
for i in xrange(len(item_id_index_in_train)):
    rank_neighbor_ui.append(np.where(P_n_sort[user_id_index_in_train[i]][::-1] == item_id_index_in_train[i])[0][0])

In [70]:
rank_neighbor_ui = np.true_divide(rank_neighbor_ui, R.shape[1])

In [71]:
np.average(rank_neighbor_ui, weights=r_ui)

0.4343424788453355

In [97]:
num_bins = 1000
counts, bin_edges = np.histogram(rank_ui, bins=num_bins, density=True)
counts_p, bin_edges_p = np.histogram(rank_ui_popularity, bins=num_bins, density=True)
counts_n, bin_edges_n = np.histogram(rank_neighbor_ui, bins=num_bins, density=True)

In [100]:
import matplotlib.pyplot as plt
from scipy.interpolate import spline

In [98]:
cdf = np.true_divide(np.cumsum(counts), np.cumsum(counts)[-1])
cdf_p = np.true_divide(np.cumsum(counts_p), np.cumsum(counts_p)[-1])
cdf_n = np.true_divide(np.cumsum(counts_n), np.cumsum(counts_n)[-1])

In [106]:
plt.plot(bin_edges[1:], cdf)
plt.plot(bin_edges_p[1:], cdf_p)
plt.plot(bin_edges_n[1:], cdf_n)

plt.xlabel("top %")
plt.ylabel("probability")
plt.legend(["implicit", "popularity", "neighbor"], loc="upper left")
plt.title("Probability that a item purchased in test set falls within top x% of recommended items")

plt.show()